In [ ]:
### test chinese dataset

import csv
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Initialization
rng = np.random

# Parameters
learning_rate = 0.01
training_epochs = 25
display_step = 10

# Network Parameters
batch_size = 100
vocab_size = 10000
n_hidden_1 = 256
n_hidden_2 = 256
n_embedding = 256
positive_samples = 4

data_index = 0
window_size = 25
half_window = round((window_size - 1) / 2)
num_skips = 2
words_per_batch = round(batch_size / num_skips)


# Model itself
layer_input = tf.placeholder("float", [batch_size, vocab_size], "layer_input")
layer_output_nums = tf.placeholder(tf.int32, [batch_size, positive_samples], "layer_output")

w_l1 = tf.Variable(tf.random_normal([vocab_size, n_hidden_1]))
w_l1_bias = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1_sum = tf.add(tf.matmul(layer_input, w_l1), w_l1_bias)
layer_1 = tf.nn.relu(layer_1_sum)


w_l2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
w_l2_bias = tf.Variable(tf.random_normal([n_hidden_2]))
layer_2_sum = tf.add(tf.matmul(layer_1, w_l2), w_l2_bias)

w_out = tf.Variable(tf.random_normal([vocab_size, n_hidden_2]))
w_out_bias = tf.Variable(tf.random_normal([vocab_size]))

loss = tf.nn.sampled_softmax_loss(w_out, w_out_bias, layer_output_nums, layer_2_sum, 
                                  num_sampled = batch_size, num_classes = vocab_size, num_true = positive_samples)
cost = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

init = tf.global_variables_initializer()
train_inputs = tf.placeholder(tf.float64, shape=[batch_size, vocab_size], name = "train_inputs")

In [87]:
## prepare data
import json
data_index = 0

## get words
def gen_word_set(file_path):
    word_set = set()
    with open(file_path, encoding='utf8') as f:
        for line in f.readlines():
            spline = line.strip().split('\t')
            if len(spline) < 4:
                continue
            prefix, query_pred, title, tag, label = spline
            if label == '0':
                continue
            cur_arr = [prefix, title]
            query_pred = json.loads(query_pred)
            for w in prefix:
                word_set.add(w)
            for each in query_pred:
                for w in each:
                    word_set.add(w)
    return list(word_set)

##
file_vali = 'oppo_round1_train_20180929_mini.txt'
chi_wordset= gen_word_set(file_path=file_vali)
print(type(chi_wordset))

<class 'list'>


In [90]:
from unicodedata import numeric

chi_wordset=[]
for i in chi_wordset:
    res = i.encode().decode('utf-8')
    chi_wordset.append(res)

In [91]:
from collections import Counter, defaultdict
from tqdm import tqdm

def explode_word(word):
    dictionary = defaultdict(lambda: 0)
    w = "#" + word + "#"
    for i in range(0, len(w)-3+1):
        yield w[i:i+3]
##
def word2tri(words):
    count=Counter()
    for w in tqdm(words):
        count.update(explode_word(w))
    count = [["###", -1]] + count.most_common(vocab_size - 1)
    dictionary = defaultdict(lambda: 0)
    for key, val in enumerate(count):
        dictionary[val[0]] = key
    return list(dictionary.keys())

tri_gram = word2tri(chi_wordset)
print(type(tri_gram))

0it [00:00, ?it/s]

<class 'list'>


In [73]:
# from __future__ import unicode_literals
# from unicodedata import numeric

# def to_integer(s):
#     try:
#         r = int(s)
#     except ValueError:
#         r = int(numeric(s))
#     return r

# print(to_integer('#埔#'))
    
    

In [75]:
def generate_batch():
    global data_index
    buffer_x = []
    buffer_y = []
    for _ in range(words_per_batch):
        x = [0] * vocab_size
        for _ in range(num_skips):
            y = [0] * positive_samples
            sample_word = rng.randint(-half_window, half_window - 1)
            if half_window >= 0: 
                sample_word += 1 
            words_ranges= word2tri(chi_wordset[data_index+half_window+ sample_word])
            #sample_tris = list(word2tri(chi_wordset[data_index + half_window + sample_word]))
            for i, k in enumerate(rng.choice(list(words_ranges), positive_samples)):
                y[i] = k
            buffer_x.append(x)
            buffer_y.append(y)
        data_index = (data_index + 1) % (vocab_size - window_size)
    return buffer_x, buffer_y
## generate X, Y batch
# X, Y = generate_batch()

In [76]:
np.set_printoptions(precision=3)
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        cc = []
        for _ in range(round((vocab_size - window_size) / words_per_batch)):
            batch_x, batch_y = generate_batch()
            _, c = sess.run([optimizer, cost], feed_dict={layer_input: batch_x, layer_output_nums: batch_y})
            cc.append(c)
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(np.mean(np.array(cc))))

IndexError: list index out of range